---
title: "Keras regression model"
subtitle: "with whiskynote.be data"
author: "Tony Duan"

execute:
  warning: false
  error: false

format:
  html:
    toc: true
    toc-location: right
    code-fold: show
    code-tools: true
    number-sections: true
    code-block-bg: true
    code-block-border-left: "#31BAE9"
---


In [ ]:
import tensorflow as tf

import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns

from siuba.siu import call
from siuba import _, mutate, filter, group_by, summarize,show_query
from siuba import *

from siuba.data import mtcars,penguins

# read in data

In [ ]:
import pandas as pd
data=pd.read_excel('./output/all_page_bottle_list_all.xlsx')

In [ ]:
list(data)

In [ ]:
data.info()

In [ ]:
import re
data001=data>> filter(_.all_page_score >=70
                      ,_.all_page_score <100
                      ,_.bottle_review_Nose !='no comment'
                      ,_.bottle_review_Mouth !='no comment'
                      ,_.bottle_review_Finish !='no comment'
                      ) >>mutate(
                      review=_.bottle_name+_.bottle_review_Nose+_.bottle_review_Mouth+_.bottle_review_Finish
                      )>>mutate(review=_.review.str.lower().str.replace('nose:','').str.replace('mouth:','').str.replace('finish:','').str.replace('.','').str.replace(',','').str.replace('(','').str.replace(')','').str.replace('-','').str.replace('apples','apple').str.replace('oranges','orange').str.replace('sweetness','sweet').str.replace('fruits','fruit'))>>mutate(review_len=_.review.str.count(' ') + 1)


In [ ]:
data001['review_flag']= np.where(data001['all_page_score']>=90, 1, 0)

# shuffle data

In [ ]:
data002=data001.sample(frac=1)

In [ ]:
data002.to_excel('data002.xlsx')

In [ ]:
data002.info()

In [ ]:
review=data002['review'].tolist()

In [ ]:
review[2]

In [ ]:
review_flag=data002["review_flag"].tolist()

In [ ]:
review_score=data002["all_page_score"].tolist()

In [ ]:
review_flag[2]

In [ ]:
from collections import Counter
Counter(review_flag)

In [ ]:
print(len(review))

print(len(review_flag))

# transfer data

In [ ]:
import tensorflow as tf
import numpy as np 
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# Initialize the Tokenizer class
tokenizer = Tokenizer()

# Generate the word index dictionary
tokenizer.fit_on_texts(review)

# Define the total words. You add 1 for the index `0` which is just the padding token.
total_words = len(tokenizer.word_index) + 1

In [ ]:
print(f'total words: {total_words}')

In [ ]:
# Convert labels lists to numpy array
review_flag_final = np.array(review_flag)
review_score_final = np.array(review_score)

In [ ]:
# Parameters
vocab_size = 7000
max_length = 300
embedding_dim = 16
#trunc_type='pre'
trunc_type='post'
oov_tok = "<OOV>"

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Initialize the Tokenizer class
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)

# Generate the word index dictionary for the training sentences
tokenizer.fit_on_texts(review)
word_index = tokenizer.word_index

# Generate and pad the training sequences
sequences = tokenizer.texts_to_sequences(review)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

In [ ]:
len(review[4])

In [ ]:
len(padded[4])

In [ ]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])            

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

In [ ]:
print(decode_review(padded[4]))

after tokenizer

In [ ]:
print(sequences[4])

In [ ]:
review_flag[4]

# using 4000 to train and 633 to test

In [ ]:
padded_train=padded[0:4000]
padded_test=padded[4000:]

In [ ]:
review_flag_final_train=review_flag_final[0:4000]
review_flag_final_test=review_flag_final[4000:]

In [ ]:
review_socre_final_train=review_score_final[0:4000]
review_socre_final_test=review_score_final[4000:]

## total

In [ ]:
len(padded)
len(review_flag_final)

## train

In [ ]:
len(padded_train)
len(review_flag_final_train)

## test

In [ ]:
len(padded_test)
len(review_flag_final_test)

In [ ]:
sum(review_flag_final_test)

## if all guess lower than 90 points then 0.72 accuracy 

In [ ]:
(len(review_flag_final_test)-sum(review_flag_final_test))/len(review_flag_final_test)

# dummy model

trainning score average is 86.5

In [ ]:
sum(review_socre_final_train)/len(review_socre_final_train)

In [ ]:
import numpy as np
sum(np.absolute(86.5-review_socre_final_test))/len(review_socre_final_test)

# DNN regression model 


In [ ]:
# Build the model
model_dnn= tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size,output_dim=32),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [ ]:
# Initialize the optimizer
optimizer = tf.keras.optimizers.Adam(0.001)

# Set the training parameters
model_dnn.compile(loss=tf.keras.losses.Huber(), optimizer=optimizer, metrics=["mae"])

In [ ]:
model_dnn.summary()

## train model

In [ ]:
# Train the model
history = model_dnn.fit(x=padded_train, y=review_socre_final_train,validation_data=(padded_test, review_socre_final_test),epochs=200,verbose=0 )

#history = model_dnn.fit(x=padded_train, y=review_socre_final_train,validation_split=0.2,epochs=20)

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']

mae = history.history['mae']
val_mae = history.history['val_mae']

epochs = range(len(val_loss))

In [ ]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('DNN model Training and validation loss')
plt.legend()

plt.show()

In [ ]:
# import matplotlib.image as mpimg
# import matplotlib.pyplot as plt
# #------------------------------------------------
# # Plot training and validation loss per epoch
# #------------------------------------------------
# 
# plt.plot(epochs, mae, 'r', label='Training mae')
# plt.plot(epochs, val_mae, 'b', label='Validation mae')
# plt.title('DNN model Training and validation mae')
# plt.legend()
# 
# plt.show()

In [ ]:
# Only plot the last 80% of the epochs
zoom_split = int(epochs[-1] * 0.2)
epochs_zoom = epochs[zoom_split:]
val_loss_zoom = val_loss[zoom_split:]
loss_zoom = loss[zoom_split:]

# Plot zoomed mae and loss
plt.plot(epochs_zoom, loss_zoom, 'r', label='Training Loss')
plt.plot(epochs_zoom, val_loss_zoom, 'b', label='Validation Loss')
plt.title('DNN model Training and validation loss')
plt.legend()

plt.show()

## predication

In [ ]:
x = padded_test
y = model_dnn.predict(x)

In [ ]:
len(padded_test)
len(y)
len(review_socre_final_test)

In [ ]:
review_socre_final_test.shape

In [ ]:
y.shape

In [ ]:
y2 = y.flatten()

In [ ]:
y2.shape

In [ ]:
dataset = pd.DataFrame({'real': review_socre_final_test, 'predic': list(y2)}, columns=['real', 'predic'])

In [ ]:
dataset['predic']=round(dataset['predic'])
dataset['predic']=round(dataset['predic'])

In [ ]:
dataset=dataset>> mutate(predic=if_else(_.predic <70, 70, _.predic)
                          ,dummy_pred=86
                         ,diff=_.predic-_.real 
                         ,dummy_diff=_.dummy_pred-_.real
                          )>> mutate(predic=if_else(_.predic >100,100, _.predic)
                                     ,predic_class=if_else(_.predic >=90,1, 0)
                                    ,real_class=if_else(_.real>=90,1, 0)
                                    ,dummy_class=0
                          )
                          
dataset002 = pd.concat([data002[4000:].reset_index(drop=True),dataset.reset_index(drop=True)], axis=1)                    

In [ ]:
dataset002.to_excel('pred.xlsx')

# mean absolute error 

In [ ]:
sum(abs(dataset002['diff']))/len(dataset002)

In [ ]:
sum(abs(dataset002['dummy_diff']))/len(dataset002)

In [ ]:
sum(dataset002['predic']>=90)

In [ ]:
sum(dataset002['real']>=90)

In [ ]:
from sklearn import metrics
confusion_matrix_dt = metrics.confusion_matrix(dataset002['real_class'],dataset002['predic_class']) 
confusion_matrix_dt

In [ ]:
from sklearn import metrics
accuracy = metrics.accuracy_score(dataset002['real_class'],dataset002['predic_class'])  
accuracy

In [ ]:
from sklearn import metrics
accuracy = metrics.accuracy_score(dataset002['real_class'],dataset002['dummy_class'])  
accuracy

In [ ]:
import seaborn as sns
fig, ax = plt.subplots()

sns.scatterplot(data=dataset,x='real',y='predic',ax=ax)
sns.regplot(data=dataset, x="real", y="predic", x_jitter=.15,ax=ax)
ax.set(xlim=(65, 100),ylim=(65, 100))

# save model

In [ ]:
#model_dnn.save('whiskynote_score_dnn.keras')

## load model

In [ ]:
new_model = tf.keras.models.load_model('whiskynote_score_dnn.keras')

In [ ]:
new_model.summary()

# new data

In [ ]:
import tensorflow as tf

import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns

from siuba.siu import call
from siuba import _, mutate, filter, group_by, summarize,show_query
from siuba import *

from siuba.data import mtcars,penguins

# read in data

In [ ]:
import pandas as pd
new_data=pd.read_excel('./output/all_page_bottle_list_all_new.xlsx')

In [ ]:
import re
new_data001=new_data>> filter(_.all_page_score >=70
                      ,_.all_page_score <100
                      ,_.bottle_review_Nose !='no comment'
                      ,_.bottle_review_Mouth !='no comment'
                      ,_.bottle_review_Finish !='no comment'
                      ) >>mutate(
                      review=_.bottle_name+_.bottle_review_Nose+_.bottle_review_Mouth+_.bottle_review_Finish
                      )>>mutate(review=_.review.str.lower().str.replace('nose:','').str.replace('mouth:','').str.replace('finish:','').str.replace('.','').str.replace(',','').str.replace('(','').str.replace(')','').str.replace('-','').str.replace('apples','apple').str.replace('oranges','orange').str.replace('sweetness','sweet').str.replace('fruits','fruit'))>>mutate(review_len=_.review.str.count(' ') + 1)


In [ ]:
new_data001['review_flag']= np.where(new_data001['all_page_score']>=90, 1, 0)

# shuffle data

In [ ]:
new_data001=new_data001.sample(frac=1).head(100)

In [ ]:
new_review=new_data001['review'].tolist()

In [ ]:
new_review[2]

In [ ]:
new_review_flag=new_data001["review_flag"].tolist()

In [ ]:
new_review_score=new_data001["all_page_score"].tolist()

In [ ]:
new_review_flag[2]

In [ ]:
from collections import Counter
Counter(new_review_flag)

In [ ]:
print(len(new_review))

print(len(new_review_flag))

# transfer data

In [ ]:
import tensorflow as tf
import numpy as np 
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# Initialize the Tokenizer class
tokenizer = Tokenizer()

# Generate the word index dictionary
# ! using model training ord index dictionary
tokenizer.fit_on_texts(review)

# Define the total words. You add 1 for the index `0` which is just the padding token.
total_words = len(tokenizer.word_index) + 1

In [ ]:
print(f'total words: {total_words}')

In [ ]:
# Convert labels lists to numpy array
new_review_flag_final = np.array(new_review_flag)
new_review_score_final = np.array(new_review_score)

In [ ]:
# Parameters
vocab_size = 7000
max_length = 300
embedding_dim = 16
#trunc_type='pre'
trunc_type='post'
oov_tok = "<OOV>"

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Initialize the Tokenizer class
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)

# Generate the word index dictionary for the training sentences
# ! using model training ord index dictionary
tokenizer.fit_on_texts(review)
word_index = tokenizer.word_index

# Generate and pad the training sequences 
#! on new data
new_sequences = tokenizer.texts_to_sequences(new_review)
new_padded = pad_sequences(new_sequences,maxlen=max_length, truncating=trunc_type)

In [ ]:
len(new_review[4])

In [ ]:
len(new_padded[4])

In [ ]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])            

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

In [ ]:
print(decode_review(new_padded[4]))

after tokenizer

In [ ]:
print(new_sequences[4])

In [ ]:
new_review_flag[4]

# predication

In [ ]:
x = new_padded
y = new_model.predict(x)

In [ ]:
len(x)
len(y)

In [ ]:
y2 = y.flatten()

In [ ]:
y2.shape

In [ ]:
dataset = pd.DataFrame({'real': new_review_score_final, 'predic': list(y2)}, columns=['real', 'predic'])

In [ ]:
dataset['predic']=round(dataset['predic'])
dataset['predic']=round(dataset['predic'])

In [ ]:
dataset=dataset>> mutate(predic=if_else(_.predic <70, 70, _.predic)
                          ,dummy_pred=86
                         ,diff=_.predic-_.real 
                         ,dummy_diff=_.dummy_pred-_.real
                          )>> mutate(predic=if_else(_.predic >100,100, _.predic)
                                     ,predic_class=if_else(_.predic >=90,1, 0)
                                    ,real_class=if_else(_.real>=90,1, 0)
                                    ,dummy_class=0
                          )
                          
dataset002 = pd.concat([data002.head(100).reset_index(drop=True),dataset.reset_index(drop=True)], axis=1)                    

# mean absolute error 

In [ ]:
sum(abs(dataset002['diff']))/len(dataset002)

In [ ]:
sum(abs(dataset002['dummy_diff']))/len(dataset002)

In [ ]:
sum(dataset002['predic']>=90)

In [ ]:
sum(dataset002['real']>=90)

In [ ]:
from sklearn import metrics
confusion_matrix_dt = metrics.confusion_matrix(dataset002['real_class'],dataset002['predic_class']) 
confusion_matrix_dt

In [ ]:
from sklearn import metrics
accuracy = metrics.accuracy_score(dataset002['real_class'],dataset002['predic_class'])  
accuracy

In [ ]:
from sklearn import metrics
accuracy = metrics.accuracy_score(dataset002['real_class'],dataset002['dummy_class'])  
accuracy

In [ ]:
import seaborn as sns
fig, ax = plt.subplots()

sns.scatterplot(data=dataset,x='real',y='predic',ax=ax)
sns.regplot(data=dataset, x="real", y="predic", x_jitter=.15,ax=ax)
ax.set(xlim=(65, 100),ylim=(65, 100))

# resource:

https://www.tensorflow.org/tutorials/keras/regression
